In [ ]:
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options 
import time
from bs4 import BeautifulSoup
import pandas as pd
import csv
from random import randint
from selenium.webdriver.support.wait import WebDriverWait
 
url="https://www.ukdirectory.co.uk/health-and-beauty/hospitals-and-services/"
link=url
chrome_options = Options()  
chrome_options.add_argument("--headless")  
driver =webdriver.Chrome(executable_path=r'C:\Users\DELL\.wdm\drivers\chromedriver\80.0.3987.106\win32\chromedriver.exe')
driver.get(url)
time.sleep(3)

cat="hospital"
country="uk"
data_link=[]
while True:
    elem_list=driver.find_elements_by_class_name("directory-listing")
    for e in elem_list:
        try:
            h3_tag=e.find_element_by_tag_name("h3")
            a_tag=h3_tag.find_element_by_tag_name("a").get_attribute("href")
            data_link.append(a_tag)
            print(a_tag)
        except Exception as e:
            print("NO A TAG",e)
            continue
    try:
        time.sleep(2)
        pagination=driver.find_element_by_class_name("paging")
        #print(pagination.text)
        ul_tag=pagination.find_element_by_tag_name("ul")
        a_tagg=ul_tag.find_elements_by_tag_name("li")[-1]
        c_elem=a_tagg.find_element_by_tag_name("a")
        a_text=c_elem.text
        print(a_text)
        #c_elem.click()
        u=c_elem.get_attribute("href")
        print(u)
        if u ==driver.current_url:
            break
        else:  
            #a_tagg.click()
            driver.get(u)
            time.sleep(3)
    except Exception as e:
        print("NO NEXT",e)
        break
head=["Name","Address","domain","source","category","place"]
#skipped_links=[]
with open('uk_directory_hospital.csv', mode='w+',encoding="UTF-8",newline="") as file:
    writer1 = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer1.writerow(head)
    for li in data_link:
        try:
            driver.get(li)
            name=None
            address=None
            time.sleep(5)
            full_block=driver.find_element_by_class_name("local-listing")
            name_block=full_block.find_element_by_class_name('companyDetails')
            nn=name_block.find_element_by_tag_name("h1").text
            if nn is not None:
                name=nn.strip()    
            
            address_block=full_block.find_element_by_tag_name("address")
            if address_block is not None:
                address=address_block.text
            block=full_block.find_element_by_class_name("contact-info")
            data=block.text
            domain=None
            if "ebsite" in data or "www" in data or "http" in data:
                if "ebsite" in data:
                    ind1=data.index("ebsite")
                    bl=data[ind1+6:]
                    ind2=bl.index(".")
                    domain=data[ind1+6:ind2+15].strip()
                elif "www" in data:
                    ind1=data.index("www")
                    domain=data[ind1:ind1+40].strip()
                elif "http" in data:
                    ind1=data.index("http")
                    bl=data[ind1:]
                    ind2=bl.index(".")
                    domain=data[ind1+4:ind2+15].strip()
            
            print(name)
            print(domain)
            print(address)
            writer1.writerow([name,address,domain,link,cat,country])
        except Exception as e:
            print("skipp",e)
            continue



